In [1]:
!pip install -U LibRecommender

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.2 MB/s eta 0:00:00


In [2]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import tensorflow as tf
import tqdm
import warnings
import fastparquet
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

2024-03-06 09:05:13.374864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 09:05:13.375005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 09:05:13.552775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def f1 (real, pred, **kwargs):
    return f1_score(real, (pred > 0.2).astype(int), **kwargs)

N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TARGET_NAME = 'label'

In [5]:
s_Train = pd.read_parquet('/kaggle/input/next-orders/s_Train (1).parquet')
# s_Test = pd.read_parquet('/kaggle/input/next-orders/s_Test (1).parquet')

s_Train.rename(columns={'user_id': 'user', 'category': 'item', 'target': 'label'}, inplace=True)
new_order = ['user', 'item', 'label'] + [col for col in s_Train.columns if col not in ['user', 'item', 'label']]
s_Train = s_Train.reindex(columns=new_order)

Train_set, Valid_set = train_test_split(s_Train, test_size = TEST_SIZE,
                                        stratify = s_Train[TARGET_NAME], random_state = 23)
Train_set.reset_index(drop=True, inplace=True)
Valid_set.reset_index(drop=True, inplace=True)

del s_Train

In [6]:
Train_set.drop(columns = ['id'], axis = 1, inplace = True)
# Valid_set.drop(columns = 'id', axis = 1, inplace = True)

In [7]:
from libreco.data import DatasetFeat

sparse_col = ["Clusters"]
dense_col = ['total_order_in_cat_max',
 'ordered_mean',
 'pc2',
 'total_rating_cat_max',
 'rating_max',
 'rating_mean',
 'pc5',
 'weekday_mean',
 'pc6',
 'pc4',
 'pc10',
 'pc3',
 'pc1',
 'pc7',
 'pc8',
 'pc9',
 'rating_per_month_mean',
 'month_std',
 'month_mean',
 'rating_per_month_std',
 'weekday_std',
 'month_max',
 'ordered_std',
 'rating_std',
 'weekday_min',
 'month_min',
 't_rating_per_month_mean',
 'Clusters',
 'weekday_max',
 't_rating_per_month_sum',
 'tc_rating_per_month_mean',
 't_rating_per_month_median',
 't_rating_per_month_std',
 'tc_rating_per_month_sum',
 'ordered_max',
 'tc_rating_per_month_std']
user_col = []
item_col = ['Clusters', 'total_order_in_cat_max',
 'ordered_mean',
 'pc2',
 'total_rating_cat_max',
 'rating_max',
 'rating_mean',
 'pc5',
 'weekday_mean',
 'pc6',
 'pc4',
 'pc10',
 'pc3',
 'pc1',
 'pc7',
 'pc8',
 'pc9',
 'rating_per_month_mean',
 'month_std',
 'month_mean',
 'rating_per_month_std',
 'weekday_std',
 'month_max',
 'ordered_std',
 'rating_std',
 'weekday_min',
 'month_min',
 't_rating_per_month_mean',
 'Clusters',
 'weekday_max',
 't_rating_per_month_sum',
 'tc_rating_per_month_mean',
 't_rating_per_month_median',
 't_rating_per_month_std',
 'tc_rating_per_month_sum',
 'ordered_max',
 'tc_rating_per_month_std']

In [8]:
train_data, data_info = DatasetFeat.build_trainset(Train_set, user_col, item_col, sparse_col, dense_col)
eval_data = DatasetFeat.build_evalset(Valid_set)
# test_data = DatasetFeat.build_testset(test_data)

In [9]:
from libreco.algorithms import DeepFM

model = DeepFM(
    task="ranking",
    data_info=data_info,
    embed_size=16,
    n_epochs=2,
    use_bn=True,
    hidden_units=(128, 64, 32),
)

model.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall"],
)

Training start time: 2024-03-06 09:08:43
total params: 447,783 | embedding params: 355,951 | network params: 91,832


train: 100%|██████████| 110125/110125 [17:54<00:00, 102.53it/s]


Epoch 1 elapsed: 1074.116s
	 train_loss: 1.5871


eval_listwise: 100%|██████████| 1931/1931 [01:21<00:00, 23.78it/s]


	 eval log_loss: 0.6932
	 eval roc_auc: 0.5001
	 eval precision@10: 0.0993
	 eval recall@10: 0.3122


train: 100%|██████████| 110125/110125 [17:48<00:00, 103.04it/s]


Epoch 2 elapsed: 1068.777s
	 train_loss: 0.6937


eval_listwise: 100%|██████████| 1931/1931 [01:21<00:00, 23.57it/s]


	 eval log_loss: 0.6933
	 eval roc_auc: 0.4997
	 eval precision@10: 0.0642
	 eval recall@10: 0.2011


In [10]:
data_info.save("./", model_name="wide_deep")
model.save("./", model_name="wide_deep")